<a href="https://colab.research.google.com/github/adarsh-meher/Kaggle---Predict-Real-disaster-tweets/blob/master/Kaggle_Real_or_not_Disaster_Tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://github.com/adarsh-meher/Kaggle---Predict-Real-disaster-tweets.git
import os
os.chdir('/content/Kaggle---Predict-Real-disaster-tweets/')

Cloning into 'Kaggle---Predict-Real-disaster-tweets'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [14]:
os.listdir(os.getcwd())

['README.md',
 'sample_submission.csv',
 'Kaggle---Predict-Real-disaster-tweets',
 '.git',
 'test.csv',
 'train.csv']

In [15]:
!pip install fasttext
!pip install fastai

In [0]:
import pandas as pd
import numpy as np
import dask as D
import scipy as sc
import spacy as S
import gensim as G
import torch as T
import fasttext as ft
import fastai as fai
import hyperopt as hopt

In [0]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")